In [200]:
import pickle
import random

import numpy as np

matches_df = pickle.load(open("database/matches_df.pck", "rb"))

In [201]:
winner_list = []
market_val_diff = []
goals_diff = []
for _, row in matches_df.iterrows():
    if row["Team Home ID"] == row["Winner Team ID"]:
        winner_list.append("Home")
    elif row["Team Away ID"] == row["Winner Team ID"]:
        winner_list.append("Away")
    elif row["Winner Team ID"] == 0:
        winner_list.append("Draw")
    market_val_diff.append(row["Market Value Home"] - row["Market Value Away"])
    goals_diff.append(row["Goals Home"] - row["Goals Away"])
matches_df["Result"] = winner_list
matches_df["Market Value Difference"] = market_val_diff
matches_df["Goals Difference"] = goals_diff

In [211]:
from tqdm import tqdm
import numpy as np

from sklearn.model_selection import train_test_split



def opt(interval, train_df, df, type):
    last = 900
    range_dicts_exact = dict()
    for i in range(700, -900, -interval):
        filtered_df = train_df[(train_df["Market Value Difference"] > i) & (train_df["Market Value Difference"] < last)]
        if len(filtered_df) == 0:
            pass
        else:
            range_dicts_exact[str(last) + "," + str(i)] = filtered_df.groupby(by=["Goals Home", "Goals Away"]).size()/len(filtered_df)
            last = i

    range_dict_exact_keys = range_dicts_exact.keys()
    y_pred = []
    val_i = 0
    row_i = 0
    for _, row in df.iterrows():
        for key in range_dict_exact_keys:
            interval = [int(x) for x in key.split(",")]
            if interval[0] > row["Market Value Difference"] > interval[1]:
                pred_res = range_dicts_exact[key].idxmax()
                pred_res = (pred_res[0], pred_res[1], pred_res[0] - pred_res[1])
                y_pred.append(pred_res)
                val_i += 1
                pass
        row_i += 1
        if val_i != row_i:
            if row["Market Value Difference"] > 0:
                pred_res = (2, 1 , 1)
            else:
                pred_res = (1, 2, -1)
            y_pred.append(pred_res)
            val_i += 1

    points = 0
    point_list = []
    for index, (_, row) in enumerate(df.iterrows()):
        pred_home_goals, pred_away_goals, pred_goal_diff = y_pred[index]
        if pred_home_goals == row["Goals Home"] and pred_away_goals == row["Goals Away"]:
            points += 5
            point_list.append(5)
        elif pred_goal_diff == row["Goals Difference"]:
            points += 3
            point_list.append(3)
        elif (pred_goal_diff > 0 and row["Goals Difference"] > 0) or (pred_goal_diff < 0 and row["Goals Difference"] < 0):
            points += 1
            point_list.append(1)
        else:
            point_list.append(0)
    if type == "points":
        points *= 306 / len(df)
        return [points, 0]
    else:
        return [3 * point_list.count(3.0) + 2 * point_list.count(5.0) + point_list.count(1.0), points]


def try_out(train_df, test_df, validate_df):
    opt_list = []
    for i in range(30, 600, 10):
        opt_list.append([i, opt(i, train_df, test_df, "")])
    opt_list = sorted(opt_list, key= lambda x: x[1][0], reverse=True)
    return opt(opt_list[0][0], train_df, validate_df, "points")

resulted_points = []

for _ in tqdm(range(500)):
    train_df, test_df = train_test_split(matches_df, test_size=612)
    test_df, validate_df = train_test_split(test_df, test_size=306)
    resulted_points.append(try_out(train_df, test_df, validate_df))


100%|██████████| 500/500 [10:10<00:00,  1.22s/it]


In [212]:
resulted_points = [x[0] for x in resulted_points]

In [213]:
np.mean(resulted_points), np.max(resulted_points), np.min(resulted_points), np.std(resulted_points)

(330.91, 413.0, 253.0, 28.434167826753782)